In [16]:
import numpy as np
import time
from collections import Counter, defaultdict

In [2]:
import nltk
nltk.download('brown')
from nltk.corpus import brown
nltk.download('universal_tagset')

n_sentences = len(brown.tagged_sents(tagset='universal'))

[nltk_data] Downloading package brown to /Users/taohuadao/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/taohuadao/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [10]:
X, Y = [], []
for sentence in brown.tagged_sents(tagset='universal'):
    X.append([pair[0] for pair in sentence])
    Y.append([pair[1] for pair in sentence])

print("Found %i sentences and %i instances: " % (n_sentences, sum([len(x) for x in X])))
print("Example: ")
print("sent: ", X[0])
print("tags: ", Y[0])

print("tag set: ", set([tag for sent in Y for tag in sent]))

Found 57340 sentences and 1161192 instances: 
Example: 
sent:  ['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']
tags:  ['DET', 'NOUN', 'NOUN', 'ADJ', 'NOUN', 'VERB', 'NOUN', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'VERB', '.', 'DET', 'NOUN', '.', 'ADP', 'DET', 'NOUN', 'VERB', 'NOUN', '.']
tag set:  {'ADV', 'VERB', 'ADJ', 'PRON', 'NUM', 'DET', '.', 'ADP', 'PRT', 'X', 'CONJ', 'NOUN'}


In [11]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

def tag_accuracy(y, y_hat):
    accuracy = []
    for y1, y_hat1 in zip(y, y_hat):
        accuracy.append(sum([1 for u, v in zip(y1, y_hat1) if u == v])/len(y1))
    return accuracy

# tag_accuracy([[1,2],[2,3],[3,4]],[[1,2],[1,3],[2,2]]) #1.0, 0.5, 0.0

In [80]:
class NaiveTagger:
    def __init__(self):
        """
        The most naive part-of-speech tagger, tag the word with its most common tag
        """
        self.word_top_tag_frequency = defaultdict(lambda :Counter(['Error']).most_common(1))
        self.tags = defaultdict(lambda: 0)
        
        
    def fit(self, x_train, y_train):
        word_tag_list = defaultdict(list)
        for sent, tags in zip(x_train, y_train):
            for word, tag in zip(sent, tags):
                word_tag_list[word].append(tag)
                
        for word, tag_list in word_tag_list.items():
            temp_tag_counter = Counter(tag_list)
            self.word_top_tag_frequency[word] = temp_tag_counter.most_common(1)
            for k, v in temp_tag_counter.items():
                self.tags[k] += v
        
        self.majority_vote = sorted(self.tags.items(), key = lambda kv: kv[1], reverse = True)[0][0]
        

    
    def predict(self, x):
        """
        x: a list of test sentences, every element is a list of words which represent a sentence
        """
        y_hat = []
        for sent in x:
            y_cur = []
            for word in sent:
                if word in self.word_top_tag_frequency:
                    y_cur.append(self.word_top_tag_frequency[word][0][0])
                else:
                    y_cur.append(self.majority_vote)
                    
            y_hat.append(y_cur)
        return y_hat

In [398]:
naive_tagger = NaiveTagger()
naive_tagger.fit(x_train, y_train)
# y_hat = naive_tagger.predict(x_test)

# print("Test Accuracy: ", np.mean(tag_accuracy(y_test, y_hat)))
y_hat = naive_tagger.predict(x_train)

print("Test Accuracy: ", np.mean(tag_accuracy(y_train, y_hat)))

Test Accuracy:  0.9582979021799198


In [455]:
import math
import sys
class HmmTagger:
    def __init__(self, order = 1, unk = 'majority'):
        self.order = order
        self.states = set()
        self.unk = unk
        self.state_given_nstate = defaultdict(lambda: defaultdict(lambda: -sys.maxsize-1))
        self.signal_given_nstate = defaultdict(lambda: defaultdict(lambda: -sys.maxsize-1))
        
    def fit(self, x_train, y_train):
        nstate_state_list, nstate_signal_list = defaultdict(list), defaultdict(list)
        states_list = []
        order_states_list = []
        order1_states_list = []
        for state_seq, signal_seq in zip(y_train, x_train):
            state_window = []
            for state, signal in zip(state_seq, signal_seq):
                self.states.add(state)
                states_list.append(state)
                previous_states = tuple(["*"]*(self.order - len(state_window)) + state_window)
                nstate_state_list[previous_states].append(state)
                order_states_list.append(previous_states+tuple([state]))
                order1_states_list.append(previous_states)
                state_window.append(state)
                if len(state_window) > self.order and len(state_window) != 0:
                    state_window.pop(0)
                current_states = tuple(["*"]*(self.order - len(state_window)) + state_window)
                
                nstate_signal_list[current_states].append(signal)
        
        self.list_to_freq(self.state_given_nstate, nstate_state_list)
        self.list_to_freq(self.signal_given_nstate, nstate_signal_list)
        
        self.majority_vote = Counter(states_list).most_common(1)[0][0]
        
        state_count = Counter(states_list)
        self.state_freq=dict()
        for k, v in state_count.items():
            self.state_freq[k] = v/len(states_list)
        
        order_state_count = Counter(order_states_list)
        order_state_count1 = Counter(order1_states_list)
        self.order_state_freq=dict()
        for k, v in order_state_count.items():
            self.order_state_freq[k] = v / order_state_count1[k[:-1]]

    
    def list_to_freq(self, given_nstate, nstate_list):
        for nstate, _list in nstate_list.items():
            state_counter = Counter(_list)
            amount = sum(state_counter.values())
            for state in state_counter:
                given_nstate[nstate][state] = math.log10(state_counter[state])- math.log10(amount)

    
    def initiate_viterbi(self, signal):
        res = defaultdict(lambda: 0)
        nstate = tuple(['*']*self.order)

        for state in self.states:
            current_state = nstate[1:]+tuple([state])
            transition_prob = self.state_given_nstate[nstate][state]
            emission_prob = self.signal_given_nstate[current_state][signal]
            
            if transition_prob != -sys.maxsize-1 and emission_prob!=-sys.maxsize-1:
                prob = transition_prob + emission_prob
                res[nstate+tuple([state])] = prob
                
        # the signal has never been the first signal
        if len(res) == 0:
            if self.unk == 'majority':
                res[nstate[1:]+tuple([self.majority_vote])] = 0
            else:
                for state in self.states:
                    res[nstate[1:]+tuple([state])] = math.log10(self.state_freq[state])
        return res
    
    def get_step_prob(self, state, previous_states, signal):
        transition_prob = self.state_given_nstate[previous_states][state]
        emission_prob = self.signal_given_nstate[previous_states[1:]+tuple([state])][signal]
        if transition_prob != -sys.maxsize-1 and emission_prob!=-sys.maxsize-1:
            return transition_prob+emission_prob
        else:
            return None
    
    def predict(self, x_list):
        res = []
        print(f"begin predicting {len(x_list)} samples...")
        for i, x in enumerate(x_list):
            if i%500==0:
                print(f"{i} predictions done!")
            res.append(self.predict_one(x))
        return res
    
    def predict_one(self, x):
        '''
        viterbi是一个列表，长度为signal list的长度，每个列表元素记录一个字典，字典键是当前序列（n+1个state），值是对应的概率
        那么viterbi的键值对是怎么得到的呢？
        第一步，对于每个state，前n个state是固定的，为[‘*’]*self.order，这个时候有多少state，viterbi[0]就有多少项（除去概率为0）
        第二步，对于每个state，都有viterbi[0]中的键那么多个前序state，先把候选的n+1state的概率，选概率最高的那个，加到viterbi[1]中
        第三部，以此类推
        
        ptr也是一个列表，长度为signal list的长度 - 1， 每个元素记录一个字典，字典键是当前state，值为前n个state，
        由viterbi[i]的键得到，其中每个键后n个state为键，前n个为值
        '''
        viterbi, ptr = [defaultdict(lambda:0) for _ in range(len(x))], []
        for i, signal in enumerate(x):
            if i == 0:
                viterbi[i] = self.initiate_viterbi(x[0])
                continue
                
            ptr_layer = dict()
            for state in self.states:
                signal_prob_all = dict()
                for prev_state in viterbi[i-1]: #prev_state contains n+1 states
                    cur_state = prev_state[1:]+tuple([state])
                    
                    #prev_state:t0,t1,t2,t3,当前是t4，得到的是t1,t2,t3,t4对应的概率
                    step_prob = self.get_step_prob(state, prev_state[1:], signal)
                    if step_prob:
                        new_prob = viterbi[i-1][prev_state] + step_prob
                        signal_prob_all[cur_state]=(new_prob, prev_state)
                        
                cur_state_probs = sorted(signal_prob_all.items(), key = lambda kv:kv[1][0], reverse = True)
                
                # 取staten|staten-k+1~staten-1对应概率最大值
                for cur_state_prob in cur_state_probs:
                    #最大值可能为多个并列第一
                    if cur_state_prob[1][0] == cur_state_probs[0][1][0]:
                        
                        k = cur_state_prob[0] # sn-k~sn
                        v = cur_state_prob[1] # (P(sn|sn-k~sn-1)*P(signal|sn-k+1~sn), sn-k~sn-1)
#                         print(k, v)
                        viterbi[i][k]=v[0]
                        ptr_layer[k] = v[1]
            
            #对于所有state，没出现过signal|sn-k+1~sn）
            if len(viterbi[i])==0:
                try:
                    max_prev_state, prob = sorted(viterbi[i-1].items(), key = lambda kv:kv[1], reverse=True)[0]
                except:
                    print(viterbi, i)
                    print(x)
                    max_prev_state, prob = sorted(viterbi[i-1].items(), key = lambda kv:kv[1], reverse=True)[0]


                if self.unk == 'majority':
                    viterbi[i][max_prev_state[1:]+tuple([self.majority_vote])] = prob
                    ptr_layer[max_prev_state[1:]+tuple([self.majority_vote])] = max_prev_state
                elif self.unk == 'uni-freq':
                    for state in self.states:
                        viterbi[i][max_prev_state[1:]+tuple([state])] = prob + math.log10(self.state_freq[state])
                        ptr_layer[max_prev_state[1:]+tuple([state])] = max_prev_state
                elif self.unk =='order-freq':
                    for state in self.states:
                        current_state = max_prev_state[1:] + tuple([state])
#                         print(current_state)
#                         print(max_prev_state)
                        if current_state in self.order_state_freq:
                            viterbi[i][current_state] = prob + math.log10(self.order_state_freq[current_state])
                            ptr_layer[current_state] = max_prev_state
                    if len(viterbi[i])==0:
                        for state in self.states:
                            viterbi[i][max_prev_state[1:]+tuple([state])] = prob + math.log10(self.state_freq[state])
                            ptr_layer[max_prev_state[1:]+tuple([state])] = max_prev_state
                
                
            ptr.append(ptr_layer)
            
        return self.get_predict_res(viterbi, ptr)

    def get_predict_res(self, viterbi, ptr):
        last_state = sorted(viterbi[-1].items(), key = lambda kv:kv[1], reverse=True)[0][0]
        res = [last_state]
        for dic in ptr[::-1]:
            res.append(dic[res[-1]])
#         print(ptr)
#         print(viterbi)
#         print([r[-1] for r in res])
        return [r[-1] for r in res[::-1]]
                    
                


In [457]:
hmm_tagger = HmmTagger(2)
hmm_tagger.fit(x_train, y_train)
y_hat = hmm_tagger.predict(x_test)
print("Test Accuracy: ", np.mean(tag_accuracy(y_test, y_hat)))

# hmm_tagger.predict_one(['Muscle', 'weakness', 'did', 'not', 'improve', ',', 'and', 'the', 'patient', 'needed', 'first', 'a', 'cane', ',', 'then', 'crutches', '.'])
# y_hat = hmm_tagger.predict(x_train)
# print("Test Accuracy: ", np.mean(tag_accuracy(y_train, y_hat)))

begin predicting 11468 samples...
0 predictions done!
500 predictions done!
1000 predictions done!
1500 predictions done!
2000 predictions done!
2500 predictions done!
3000 predictions done!
3500 predictions done!
4000 predictions done!
4500 predictions done!
5000 predictions done!
5500 predictions done!
6000 predictions done!
6500 predictions done!
7000 predictions done!
7500 predictions done!
8000 predictions done!
8500 predictions done!
9000 predictions done!
9500 predictions done!
10000 predictions done!
10500 predictions done!
11000 predictions done!
Test Accuracy:  0.9082134830576258


In [429]:
hmm_tagger = HmmTagger(1, 'uni-freq')
hmm_tagger.fit(x_train, y_train)
y_hat = hmm_tagger.predict(x_test)
print("Test Accuracy: ", np.mean(tag_accuracy(y_test, y_hat)))

begin predicting 11468 samples...
0 predictions done!
500 predictions done!
1000 predictions done!
1500 predictions done!
2000 predictions done!
2500 predictions done!
3000 predictions done!
3500 predictions done!
4000 predictions done!
4500 predictions done!
5000 predictions done!
5500 predictions done!
6000 predictions done!
6500 predictions done!
7000 predictions done!
7500 predictions done!
8000 predictions done!
8500 predictions done!
9000 predictions done!
9500 predictions done!
10000 predictions done!
10500 predictions done!
11000 predictions done!
Test Accuracy:  0.933746752629939


In [456]:
hmm_tagger = HmmTagger(1, 'order-freq')
hmm_tagger.fit(x_train, y_train)
# hmm_tagger.predict_one(['He', 'replaced', 'the', 'flashlight', 'where', 'it', 'had', 'been', 'stowed', ',', 'got', 'into', 'his', 'own', 'car', 'and', 'backed', 'it', 'out', 'of', 'the', 'garage', '.'])
y_hat = hmm_tagger.predict(x_test)
print("Test Accuracy: ", np.mean(tag_accuracy(y_test, y_hat)))

begin predicting 11468 samples...
0 predictions done!
500 predictions done!
1000 predictions done!
1500 predictions done!
2000 predictions done!
2500 predictions done!
3000 predictions done!
3500 predictions done!
4000 predictions done!
4500 predictions done!
5000 predictions done!
5500 predictions done!
6000 predictions done!
6500 predictions done!
7000 predictions done!
7500 predictions done!
8000 predictions done!
8500 predictions done!
9000 predictions done!
9500 predictions done!
10000 predictions done!
10500 predictions done!
11000 predictions done!
Test Accuracy:  0.9334757030036887


In [438]:
hmm_tagger.order_state_freq

{('DET',): 9168.083333333334,
 ('VERB',): 12216.583333333334,
 ('ADP',): 9704.833333333334,
 ('NOUN',): 18487.0,
 ('PRT',): 1987.3333333333333,
 ('ADJ',): 5611.083333333333,
 ('.',): 9889.75,
 ('PRON',): 3298.6666666666665,
 ('ADV',): 3757.75,
 ('CONJ',): 2558.8333333333335,
 ('NUM',): 991.75,
 ('X',): 88.08333333333333}

In [371]:
Noun代替 Test Accuracy:  0.9364133090163975
(开头)平均概率: 0.933587694962621
（开头）实际分布: 0.9364133090163975
    
换成实际分布：
0.933746752629939
平均概率：

train_accuracy
Test Accuracy:  0.975653961190461

['Muscle', 'weakness', 'did', 'not', 'improve', ',', 'and', 'the', 'patient', 'needed', 'first', 'a', 'cane', ',', 'then', 'crutches', '.']


In [365]:
def test_fit(order):
    x_train = [['This', 'will', 'mean', 'that'],\
               ['every', 'time', "there's", 'an', 'increase', \
                'in', 'hospital', 'rates', 'your', 'cost', 'will', 'go', \
                'up', 'in', 'like', 'manner', '.']]
    y_train = [['DET', 'VERB', 'VERB', 'ADP'],
               ['DET', 'NOUN', 'PRT', 'DET', 'NOUN', \
                'ADP', 'NOUN', 'NOUN', 'DET', 'NOUN', 'VERB', \
                'VERB', 'PRT', 'ADP', 'ADJ', 'NOUN', '.']]
    hmm_tagger = HmmTagger(order)
    hmm_tagger.fit(x_train, y_train)
    print(hmm_tagger.state_given_nstate)
    print(hmm_tagger.signal_given_nstate)
    hmm_tagger.predict(['This', '0', 'mean', 'that'])
    print(hmm_tagger.majority_vote)

test_fit(2)

defaultdict(<function HmmTagger.__init__.<locals>.<lambda> at 0x7fcfe609c1f0>, {('*', '*'): defaultdict(<function HmmTagger.__init__.<locals>.<lambda>.<locals>.<lambda> at 0x7fcfe609c8b0>, {'DET': 0.0}), ('*', 'DET'): defaultdict(<function HmmTagger.__init__.<locals>.<lambda>.<locals>.<lambda> at 0x7fcfe609cb80>, {'VERB': -0.3010299956639812, 'NOUN': -0.3010299956639812}), ('DET', 'VERB'): defaultdict(<function HmmTagger.__init__.<locals>.<lambda>.<locals>.<lambda> at 0x7fcfe609c3a0>, {'VERB': 0.0}), ('VERB', 'VERB'): defaultdict(<function HmmTagger.__init__.<locals>.<lambda>.<locals>.<lambda> at 0x7fcfe609c790>, {'ADP': -0.3010299956639812, 'PRT': -0.3010299956639812}), ('DET', 'NOUN'): defaultdict(<function HmmTagger.__init__.<locals>.<lambda>.<locals>.<lambda> at 0x7fcfe609c550>, {'PRT': -0.47712125471966244, 'ADP': -0.47712125471966244, 'VERB': -0.47712125471966244}), ('NOUN', 'PRT'): defaultdict(<function HmmTagger.__init__.<locals>.<lambda>.<locals>.<lambda> at 0x7fcfe609c0d0>, {

In [366]:
hmm_tagger = HmmTagger(1)
hmm_tagger.fit(x_train, y_train)
# list(hmm_tagger.signal_given_nstate.items())[5]
# list(hmm_tagger.signal_given_nstate.items())[5]

In [358]:
hmm_tagger.majority_vote

AttributeError: 'HmmTagger' object has no attribute 'majority_vote'

In [368]:
hmm_tagger.predict(['This', 'unkmmmm', 'mean', 'that', 'every', 'time', "there's", 'an', 'increase', 'in', 'hospital', 'rates', 'your', 'cost', 'will', 'go', 'up', 'in', 'like', 'manner', '.'])

calculate one probability: nstate ('*',), signal This
state PRON
current_state ('PRON',)
state VERB
current_state ('VERB',)
state ADJ
current_state ('ADJ',)
state ADV
current_state ('ADV',)
state NUM
current_state ('NUM',)
state DET
current_state ('DET',)
state .
current_state ('.',)
state ADP
current_state ('ADP',)
state PRT
current_state ('PRT',)
state X
current_state ('X',)
state CONJ
current_state ('CONJ',)
state NOUN
current_state ('NOUN',)
initial viterbi [defaultdict(<function HmmTagger.viterbi_inc.<locals>.<lambda> at 0x7fcfe60a9700>, {('*', 'DET'): -2.7333364650857783}), defaultdict(<function HmmTagger.predict.<locals>.<listcomp>.<lambda> at 0x7fcfe60868b0>, {}), defaultdict(<function HmmTagger.predict.<locals>.<listcomp>.<lambda> at 0x7fcfe6086670>, {}), defaultdict(<function HmmTagger.predict.<locals>.<listcomp>.<lambda> at 0x7fcfe60861f0>, {}), defaultdict(<function HmmTagger.predict.<locals>.<listcomp>.<lambda> at 0x7fcfe60864c0>, {}), defaultdict(<function HmmTagger.predi

-1.1325245477634582 -9223372036854775808 ('ADP',) increase
-1.1325245477634582 -9223372036854775808 ('ADP',) increase
-2.7109907560424835 -9223372036854775808 ('PRT',) increase
-2.7109907560424835 -9223372036854775808 ('PRT',) increase
-2.120162472641768 -9223372036854775808 ('PRT',) increase
-2.120162472641768 -9223372036854775808 ('PRT',) increase
-1.605254130378586 -9223372036854775808 ('PRT',) increase
-1.605254130378586 -9223372036854775808 ('PRT',) increase
-2.842747442437252 -9223372036854775808 ('X',) increase
-2.842747442437252 -9223372036854775808 ('X',) increase
-0.3047507707664372 -9223372036854775808 ('X',) increase
-0.3047507707664372 -9223372036854775808 ('X',) increase
-3.2319365966148808 -9223372036854775808 ('X',) increase
-3.2319365966148808 -9223372036854775808 ('X',) increase
-3.2025554386544193 -9223372036854775808 ('CONJ',) increase
-3.2025554386544193 -9223372036854775808 ('CONJ',) increase
-1.5760944282914922 -9223372036854775808 ('CONJ',) increase
-1.576094428

-0.6197677026709201 -9223372036854775808 ('ADJ',) cost
-1.7605986009980077 -9223372036854775808 ('ADV',) cost
-1.7605986009980077 -9223372036854775808 ('ADV',) cost
-2.0112097440349235 -9223372036854775808 ('NUM',) cost
-2.0112097440349235 -9223372036854775808 ('NUM',) cost
-2.223178635777719 -9223372036854775808 ('DET',) cost
-2.223178635777719 -9223372036854775808 ('DET',) cost
-1.8891161850086182 -9223372036854775808 ('.',) cost
-1.8891161850086182 -9223372036854775808 ('.',) cost
-2.0462082377944952 -9223372036854775808 ('ADP',) cost
-2.0462082377944952 -9223372036854775808 ('ADP',) cost
-2.7109907560424835 -9223372036854775808 ('PRT',) cost
-2.7109907560424835 -9223372036854775808 ('PRT',) cost
-2.842747442437252 -9223372036854775808 ('X',) cost
-2.842747442437252 -9223372036854775808 ('X',) cost
-3.2025554386544193 -9223372036854775808 ('CONJ',) cost
-3.2025554386544193 -9223372036854775808 ('CONJ',) cost
-0.20326725757050834 -3.1937593428857496 ('NOUN',) cost
-0.2032672575705083

-2.3204013997317263 -9223372036854775808 ('NUM',) like
-9223372036854775808 -9223372036854775808 ('NUM',) like
-9223372036854775808 -9223372036854775808 ('NUM',) like
-2.108287750889561 -9223372036854775808 ('NUM',) like
-2.108287750889561 -9223372036854775808 ('NUM',) like
-0.3422280488101217 -9223372036854775808 ('DET',) like
-0.3422280488101217 -9223372036854775808 ('DET',) like
-1.0835018911488623 -9223372036854775808 ('DET',) like
-1.0835018911488623 -9223372036854775808 ('DET',) like
-2.245101209250068 -9223372036854775808 ('DET',) like
-2.245101209250068 -9223372036854775808 ('DET',) like
-1.8115640058421043 -9223372036854775808 ('DET',) like
-1.8115640058421043 -9223372036854775808 ('DET',) like
-2.003561238076776 -9223372036854775808 ('.',) like
-2.003561238076776 -9223372036854775808 ('.',) like
-1.1106042841841113 -9223372036854775808 ('.',) like
-1.1106042841841113 -9223372036854775808 ('.',) like
-0.5475812713092818 -9223372036854775808 ('.',) like
-0.5475812713092818 -922

['DET',
 'NOUN',
 'VERB',
 'ADP',
 'DET',
 'NOUN',
 'PRT',
 'DET',
 'NOUN',
 'ADP',
 'NOUN',
 'NOUN',
 'DET',
 'NOUN',
 'VERB',
 'VERB',
 'PRT',
 'ADP',
 'ADP',
 'NOUN',
 '.']

In [209]:
print(x_train[0], y_train[0])

['This', 'will', 'mean', 'that', 'every', 'time', "there's", 'an', 'increase', 'in', 'hospital', 'rates', 'your', 'cost', 'will', 'go', 'up', 'in', 'like', 'manner', '.'] ['DET', 'VERB', 'VERB', 'ADP', 'DET', 'NOUN', 'PRT', 'DET', 'NOUN', 'ADP', 'NOUN', 'NOUN', 'DET', 'NOUN', 'VERB', 'VERB', 'PRT', 'ADP', 'ADJ', 'NOUN', '.']
